# version 1 (not use)

In [1]:
!pip install moviepy moviepy.editor  SpeechRecognition librosa numpy ffmpeg-python

ERROR: Could not find a version that satisfies the requirement moviepy.editor (from versions: none)
ERROR: No matching distribution found for moviepy.editor


In [2]:
!pip install openai-whisper pythainlp nltk matplotlib google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 33.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 59.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=10427cbf4b16aaa136e7f7ac596e8bd4b729a51c2914c73fb81edeb5f21a3ac8
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [3]:
!pip install --upgrade google-generativeai

In [4]:
import os
import re
import wave
import numpy as np
import librosa
from moviepy.editor import VideoFileClip
import whisper
from nltk.probability import FreqDist
from pythainlp.tokenize import word_tokenize
import matplotlib.pyplot as plt
import google.generativeai as genai

/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [1]:
# --- Setup Cell (เวอร์ชันแก้ไข AttributeError) ---

# 1. บังคับอัปเกรด library ทั้งหมดให้เป็นเวอร์ชันล่าสุด
!pip install --upgrade openai-whisper moviepy librosa pythainlp nltk google-generativeai

# 2. Install Thai fonts directly into the system
!apt-get update -qq
!apt-get install -y fonts-thai-tlwg

# 3. Rebuild Matplotlib's font cache (วิธีใหม่ที่ถูกต้อง)
import matplotlib
import os
import glob

print("\nClearing and rebuilding font cache...")
try:
    # หาไฟล์ cache ของฟอนต์ (ชื่อมักจะเป็น fontlist-vXXX.json)
    cache_dir = matplotlib.get_cachedir()
    for file in glob.glob(f"{cache_dir}/fontlist-*.json"):
        os.remove(file)
        print(f"Removed font cache file: {file}")
except Exception as e:
    print(f"Could not clear font cache: {e}")

print("\n✅ Setup complete! You are now ready to run the main analysis code.")
print("‼️ IMPORTANT: Please RESTART THE RUNTIME NOW before proceeding. (Runtime > Restart runtime)")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-tlwg-garuda fonts-tlwg-garuda-ttf fonts-tlwg-kinnari
  fonts-tlwg-kinnari-ttf fonts-tlwg-laksaman fonts-tlwg-laksaman-ttf
  fonts-tlwg-loma fonts-tlwg-loma-ttf fonts-tlwg-mono fonts-tlwg-mono-ttf
  fonts-tlwg-norasi fonts-tlwg-norasi-ttf fonts-tlwg-purisa
  fonts-tlwg-purisa-ttf fonts-tlwg-sawasdee fonts-tlwg-sawasdee-ttf
  fonts-tlwg-typewriter fonts-tlwg-typewriter-ttf fonts-tlwg-typist
  fonts-tlwg-typist-ttf fonts-tlwg-typo fonts-tlwg-typo-ttf fonts-tlwg-umpush
  fonts-tlwg-umpush-ttf fonts-tlwg-waree fonts-tlwg-waree-ttf
The following NEW packages will be installed:
  fonts-thai-tlwg fonts-tlwg-garuda fonts-tlwg-garuda-ttf fonts-tlwg-kinnar

In [2]:
def analyze_voice(audio_path):
    print("🎙️  Analyzing Voice with Whisper (using GPU, this will be fast)...")
    try:
        # ใช้โมเดล large-v3 เพื่อความแม่นยำสูงสุดบน GPU
        model = whisper.load_model("large-v3")
        result = model.transcribe(audio_path, word_timestamps=True)
        full_text = result['text']
        print("   - Whisper transcription successful.")
    except Exception as e:
        print(f"   - Whisper transcription failed: {e}")
        return None
    # ... (ส่วนที่เหลือของฟังก์ชันเหมือนเดิม)
    with wave.open(audio_path, 'rb') as wf:
        duration = wf.getnframes() / float(wf.getframerate())
    total_words = len(full_text.split())
    wpm = (total_words / duration) * 60 if duration > 0 else 0
    filler_words_list = ['เอ่อ', 'อ่า', 'คือ', 'แบบว่า', 'เอิ่ม', 'แบบ']
    filler_count = sum(len(re.findall(r'\b' + word + r'\b', full_text, re.IGNORECASE)) for word in filler_words_list)
    y, sr_librosa = librosa.load(audio_path)
    pitches, magnitudes = librosa.piptrack(y=y, sr=sr_librosa)
    pitch_values = pitches[magnitudes > np.median(magnitudes)]
    pitch_variation = np.std(pitch_values[pitch_values > 0]) if len(pitch_values) > 0 else 0
    print("   - Voice analysis complete.")
    return {
        "whisper_result": result, "duration": duration, "total_words": total_words,
        "wpm": wpm, "filler_count": filler_count, "pitch_variation": pitch_variation
    }

def analyze_word_variety(text):
    words = word_tokenize(text, engine="newmm")
    words = [word for word in words if word.strip()]
    ttr = len(set(words)) / len(words) if words else 0
    fdist = FreqDist(words)
    most_common_words = fdist.most_common(5)
    return {"ttr": ttr, "most_common_words": most_common_words}

In [3]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import subprocess
import sys

def setup_thai_font():
    """
    ตรวจสอบและติดตั้งฟอนต์ภาษาไทยสำหรับ Matplotlib โดยอัตโนมัติ
    """
    try:
        # ตรวจสอบว่าระบบปฏิบัติการคือ Linux (เช่น Google Colab) หรือไม่
        if "linux" in sys.platform:
            print("ระบบเป็น Linux, กำลังติดตั้งฟอนต์ผ่าน apt-get...")
            # ติดตั้งชุดฟอนต์ภาษาไทยสำหรับ Debian/Ubuntu
            subprocess.run(['apt-get', 'update', '-qq'], check=True)
            subprocess.run(['apt-get', 'install', '-y', 'fonts-thai-tlwg'], check=True)
            # เคลียร์และสร้างแคชฟอนต์ใหม่
            fm._rebuild()
            # ตั้งค่าฟอนต์ 'Loma' ซึ่งเป็นส่วนหนึ่งของ fonts-thai-tlwg
            plt.rcParams['font.family'] = 'Loma'
            print("✅ ตั้งค่าฟอนต์ 'Loma' สำหรับกราฟเรียบร้อย")
        else:
            # สำหรับ Windows หรือ OS อื่นๆ ให้ตั้งค่าเป็นฟอนต์ที่มักจะมีอยู่แล้ว
            plt.rcParams['font.family'] = 'Sarabun'
            print("✅ ตั้งค่าฟอนต์ 'Sarabun' สำหรับกราฟ (สำหรับ Windows/macOS)")

    except Exception as e:
        print(f"เกิดข้อผิดพลาดในการตั้งค่าฟอนต์อัตโนมัติ: {e}")
        print("กรุณาติดตั้งฟอนต์ภาษาไทยในระบบของคุณด้วยตนเอง")

In [4]:
def create_wpm_graph(whisper_result, duration, output_path="wpm_graph.png"):
    setup_thai_font()

    segments = whisper_result['segments']
    interval = 15
    num_intervals = int(duration // interval) + 1
    words_in_interval = [0] * num_intervals
    for segment in segments:
        for word_info in segment['words']:
            word_start_time = word_info['start']
            interval_index = int(word_start_time // interval)
            if interval_index < num_intervals:
                words_in_interval[interval_index] += 1
    wpm_intervals = [(count / interval) * 60 for count in words_in_interval]
    time_labels = [f"{i*interval}-{(i+1)*interval}s" for i in range(num_intervals)]
    plt.figure(figsize=(10, 5))
    plt.bar(time_labels, wpm_intervals, color='skyblue')
    plt.xlabel("ช่วงเวลา (วินาที)")
    plt.ylabel("Words Per Minute (WPM)")
    plt.title("ความเร็วในการพูดแต่ละช่วงเวลา")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.savefig(output_path)
    print(f"   - WPM graph saved to {output_path}")
    return output_path

In [5]:
import google.generativeai as genai

def get_generative_feedback(api_key, transcript):
    print("   - Getting generative feedback from Gemini AI...")
    try:
        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.0-pro')
        prompt = f"""
        จาก Transcript การนำเสนอด้านล่างนี้: "{transcript}"
        จงทำ 2 อย่างต่อไปนี้:
        1. **แนะนำการใช้คำให้กระชับขึ้น:** เลือกมา 2-3 ประโยคที่ดูเยิ่นเย้อ แล้วลองเขียนใหม่ให้สั้นและได้ใจความมากขึ้น
        2. **สร้างคำถามจากเนื้อหา:** ช่วยตั้งคำถามที่น่าสนใจ 3 ข้อ ที่สามารถถามผู้พูดได้จากเนื้อหาที่เขานำเสนอ
        โปรดจัดรูปแบบคำตอบให้ชัดเจน
        """
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"   - Could not get feedback from Gemini AI: {e}"

def generate_voice_feedback(voice_stats):
    good_points, points_to_improve = [], []
    if 130 <= voice_stats['wpm'] <= 180: good_points.append(f"👍 ความเร็วในการพูด ({int(voice_stats['wpm'])} WPM) อยู่ในเกณฑ์ดีมาก")
    else: points_to_improve.append(f"🏃 ความเร็วในการพูด ({int(voice_stats['wpm'])} WPM) ไม่คงที่ ควรปรับให้ใกล้เคียง 140-160 WPM")
    if voice_stats['filler_count'] <= 3: good_points.append(f"👍 มีการใช้คำฟุ่มเฟือยน้อยมาก (พบ {voice_stats['filler_count']} ครั้ง)")
    else: points_to_improve.append(f"🗣️ ควรลดการใช้คำฟุ่มเฟือย (พบ {voice_stats['filler_count']} ครั้ง)")
    if voice_stats['pitch_variation'] > 35: good_points.append("👍 โทนเสียงมีความน่าสนใจ ใช้เสียงสูง-ต่ำได้ดี")
    else: points_to_improve.append("밋 โทนเสียงค่อนข้างราบเรียบ ลองเพิ่มความหลากหลายของเสียง")
    return {"overall_score": round(((100 if 130 <= voice_stats['wpm'] <= 180 else 60) * 0.45) + (max(0, 100 - (voice_stats['filler_count'] * 5)) * 0.45) + (min(100, (voice_stats['pitch_variation'] / 35) * 100) * 0.10), 2),
            "good_points": good_points, "points_to_improve": points_to_improve}

In [6]:
# def run_voice_analysis_from_video(video_path):
#     """
#     ฟังก์ชันหลักในการรันโปรแกรม (เวอร์ชันแก้ไข)
#     """
#     if not os.path.exists(video_path):
#         print(f"Error: ไม่พบไฟล์วิดีโอที่ '{video_path}'")
#         return

#     print(f"🚀 เริ่มการวิเคราะห์ไฟล์: {os.path.basename(video_path)}")

#     audio_path = "temp_audio.wav"

#     # 1. แยกเสียงออกจากวิดีโอ (วิธีที่ปรับปรุงใหม่)
#     print("   - กำลังดึงไฟล์เสียงจากวิดีโอ...")
#     try:
#         # วิธีที่ 1: ใช้ moviepy เหมือนเดิม แต่เพิ่ม logger เพื่อดูรายละเอียด
#         video = VideoFileClip(video_path)
#         video.audio.write_audiofile(audio_path, codec='pcm_s16le', logger='bar')

#     except Exception as e:
#         print(f"   - MoviePy เกิดข้อผิดพลาด: {e}")
#         print("   - ลองใช้วิธีสำรองด้วย ffmpeg-python...")
#         try:
#             # วิธีที่ 2 (สำรอง): ใช้ ffmpeg-python ซึ่งมักจะเสถียรกว่า
#             import ffmpeg
#             (
#                 ffmpeg
#                 .input(video_path)
#                 .output(audio_path, acodec='pcm_s16le', ac=1, ar='16k')
#                 .run(overwrite_output=True, quiet=True)
#             )
#             print("   - ดึงไฟล์เสียงด้วยวิธีสำรองสำเร็จ!")
#         except Exception as ffmpeg_e:
#             print(f"   - ❌ ไม่สามารถดึงไฟล์เสียงได้ทั้ง 2 วิธี: {ffmpeg_e}")
#             # หากมีไฟล์เสียงเก่าค้างอยู่ ให้ลบทิ้ง
#             if os.path.exists(audio_path):
#                 os.remove(audio_path)
#             return

#     # 2. วิเคราะห์ไฟล์เสียง
#     voice_results = analyze_voice(audio_path)

#     # 3. ลบไฟล์เสียงชั่วคราว
#     if os.path.exists(audio_path):
#         os.remove(audio_path)

#     if voice_results is None:
#         print("\n❌ ไม่สามารถวิเคราะห์ต่อได้เนื่องจากประมวลผลเสียงไม่สำเร็จ")
#         return

#     # 4. สร้าง Feedback และแสดงผล (ส่วนนี้เหมือนเดิม)
#     final_feedback = generate_voice_feedback(voice_results)

#     print("\n" + "="*50)
#     print(" ✨ ผลการวิเคราะห์การพูด (Voice Analysis) ✨")
#     print("="*50)
#     # ... (ส่วนแสดงผลที่เหลือเหมือนเดิม) ...
#     print(f"\n🎯 คะแนนการสื่อสาร: {final_feedback['overall_score']} / 100")

#     print("\n✅ สิ่งที่ทำได้ดี (Good Points):")
#     if final_feedback['good_points']:
#         for point in final_feedback['good_points']:
#             print(f"   - {point}")
#     else:
#         print("   - (ไม่มี)")

#     print("\n🛠️ สิ่งที่ควรปรับปรุง (Areas for Improvement):")
#     if final_feedback['points_to_improve']:
#         for point in final_feedback['points_to_improve']:
#             print(f"   - {point}")
#     else:
#         print("   - ยอดเยี่ยม! ไม่มีจุดที่ต้องปรับปรุงเป็นพิเศษ")

#     print("\n📝 สคริปต์ที่พูดทั้งหมด:")
#     print(f"   \"{voice_results['text']}\"")
#     print("\n" + "="*50)

def run_voice_analysis_from_video(video_path, api_key):
    if not os.path.exists(video_path):
        print(f"Error: ไม่พบไฟล์วิดีโอที่ '{video_path}'"); return
    print(f"🚀 เริ่มการวิเคราะห์ไฟล์: {os.path.basename(video_path)}")
    audio_path = "temp_audio.wav"
    print("   - กำลังดึงไฟล์เสียงจากวิดีโอ...")
    try:
        VideoFileClip(video_path).audio.write_audiofile(audio_path, codec='pcm_s16le', logger=None)
    except Exception as e:
        print(f"   - ❌ ไม่สามารถดึงไฟล์เสียงได้: {e}"); return

    voice_results = analyze_voice(audio_path)
    os.remove(audio_path)
    if voice_results is None: print("\nวิเคราะห์ไม่สำเร็จ"); return

    transcript = voice_results['whisper_result']['text']
    word_variety_results = analyze_word_variety(transcript)
    create_wpm_graph(voice_results['whisper_result'], voice_results['duration'])
    generative_feedback = get_generative_feedback(api_key, transcript)
    final_feedback = generate_voice_feedback(voice_results)

    print("\n" + "="*60)
    print(" ✨ ผลการวิเคราะห์การพูดฉบับสมบูรณ์ (Complete Voice Analysis) ✨")
    print("="*60)
    print(f"\n⏱️ **ระยะเวลาที่ใช้:** {voice_results['duration']:.2f} วินาที")
    print(f"🎯 **คะแนนการสื่อสาร:** {final_feedback['overall_score']} / 100")
    print("\n✅ **สิ่งที่ทำได้ดี (Good Points):**"); [print(f"   - {p}") for p in final_feedback['good_points']]
    print("\n🛠️ **สิ่งที่ควรปรับปรุง (Areas for Improvement):**"); [print(f"   - {p}") for p in final_feedback['points_to_improve']]
    print("\n" + "-"*60)
    print("📊 **การวิเคราะห์เชิงลึก (In-depth Analysis):**\n")
    print(f"   - **ความหลากหลายของคำ (TTR):** {word_variety_results['ttr']:.2f} (ค่าเข้าใกล้ 1.0 หมายถึงใช้คำหลากหลาย)")
    print(f"   - **คำที่ใช้บ่อยที่สุด:** {', '.join([f'{w}({c})' for w, c in word_variety_results['most_common_words']])}")
    print(f"   - **กราฟ WPM:** ถูกบันทึกเป็นไฟล์ wpm_graph.png")
    print("\n" + "-"*60)
    print("🤖 **ข้อเสนอแนะจาก AI (Gemini Suggestions):**\n"); print(generative_feedback)
    print("\n" + "-"*60)
    print("📝 **สคริปต์ที่พูดทั้งหมด (Full Transcript):**"); print(f"   \"{transcript}\"")
    print("\n" + "="*60)

# version 2

In [3]:
!pip install faster-whisper librosa groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 105.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.1 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata

In [5]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [8]:
import os
import json
import librosa
from groq import Groq
from faster_whisper import WhisperModel

ASR_MODEL_NAME = "large-v3"
ASR_COMPUTE_TYPE = "float16" # "float16" หรือ "int8" เพื่อความเร็ว

LLM_MODEL_NAME = "llama-3.3-70b-versatile"

# Prompt เอาไว้ให้ Whisper ถอดคำฟุ่มเฟือยออกมา
PROMPT_FOR_FILLERS = """
ต่อไปนี้คือการถอดเสียงการนำเสนอแบบคำต่อคำ
กรุณาคงคำฟุ่มเฟือยทั้งหมดไว้ เช่น อ่า เอ่อ แบบว่า นะครับ นะคะ
"""

# รายการคำฟุ่มเฟือย (Filler Words) ที่ข่อยจะนับ
THAI_FILLER_WORDS = [
    "อ่า", "เอ่อ", "แบบว่า", "คือ", "แบบ", "ก็นะ",
    "ที่แบบ", "ใช่ไหม", "ใช่ป่ะ", "นะครับ", "นะคะ", "อะครับ", "อะค่ะ"
]

def transcribe_audio(file_path):
    """
    ถอดเสียงไฟล์ audio เป็น text โดยใช้ faster-whisper (Quantized)
    และใช้ Prompt เพื่อบังคับให้ถอดคำฟุ่มเฟือย
    """
    print(f"กำลังโหลดโมเดล ASR ({ASR_MODEL_NAME})...")

    model = WhisperModel(ASR_MODEL_NAME, device="cuda", compute_type=ASR_COMPUTE_TYPE)

    print(f"กำลังถอดเสียงไฟล์: {file_path}...")
    segments, info = model.transcribe(
        file_path,
        beam_size=5,
        language="th",
        initial_prompt=PROMPT_FOR_FILLERS
    )

    full_transcript = "".join([seg.text for seg in segments])

    duration_seconds = librosa.get_duration(filename=file_path)

    print("--- Transcript (ผลดิบจาก Whisper) ---")
    print(full_transcript)
    print("--------------------------------------")

    return full_transcript, duration_seconds

def analyze_presentation(transcript, duration_seconds):
    """
    วิเคราะห์ Transcript โดยใช้ LLM (Groq) เพื่อให้คะแนนและ Feedback
    """
    print("กำลังส่ง Transcript ไปให้ LLM วิเคราะห์...")

    words = transcript.split()
    total_words = len(words)
    duration_minutes = duration_seconds / 60
    words_per_minute = total_words / duration_minutes if duration_minutes > 0 else 0

    # นับคำฟุ่มเฟือย
    filler_counts = {}
    total_fillers = 0
    for filler in THAI_FILLER_WORDS:
        count = transcript.lower().count(filler)
        if count > 0:
            filler_counts[filler] = count
            total_fillers += count

    system_prompt = f"""
คุณคือโค้ชด้านการนำเสนอ (Presentation Coach) ที่เชี่ยวชาญและให้คำแนะนำที่สร้างสรรค์
หน้าที่ของคุณคือวิเคราะห์สคริปต์การนำเสนอและสถิติที่ได้รับ
และให้ผลลัพธ์กลับมาเป็น JSON Object เท่านั้น ห้ามมีข้อความอื่นใดๆ นอก JSON

รูปแบบ JSON ที่คุณต้องส่งกลับมา:
{{
  "communication_clarity_score": <float, 0-100>,
  "good_points": [
    "<string, จุดแข็งที่ 1>",
    "<string, จุดแข็งที่ 2>"
  ],
  "areas_for_improvement": [
    "<string, ข้อควรปรับปรุงที่ 1>",
    "<string, ข้อควรปรับปรุงที่ 2>"
  ],
  "statistics": {{
    "words_per_minute": <float>,
    "total_filler_words": <int>,
    "filler_word_details": <dict>
  }}
}}

เกณฑ์การให้คะแนน (communication_clarity_score):
- 100: สมบูรณ์แบบ, ชัดเจน, ไม่มีคำฟุ่มเฟือย, ความเร็วพอดี
- 85-95: ดีมาก, อาจมีคำฟุ่มเฟือยเล็กน้อย
- 70-84: ปานกลาง, พูดเร็ว/ช้าไป, มีคำฟุ่มเฟือย
- < 70: ต้องปรับปรุงด่วน, จับใจความยาก
    """

    user_prompt = f"""
ช่วยวิเคราะห์การนำเสนอครั้งนี้ให้หน่อย

--- สถิติ ---
ความเร็วในการพูด (WPM): {words_per_minute:.2f} คำต่อนาที
(เกณฑ์ WPM ที่ดีสำหรับภาษาไทย: 130-160 WPM)
จำนวนคำฟุ่มเฟือยทั้งหมด: {total_fillers} คำ
รายละเอียดคำฟุ่มเฟือย: {json.dumps(filler_counts, ensure_ascii=False)}

--- Transcript ---
{transcript}
---

โปรดวิเคราะห์และส่งผลลัพธ์เป็น JSON ตามรูปแบบที่กำหนดเท่านั้น
    """

    try:
        client = Groq()
        chat_completion = client.chat.completions.create(
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            model=LLM_MODEL_NAME,
            temperature=0.3,
            response_format={"type": "json_object"}
        )

        response_json_str = chat_completion.choices[0].message.content

        analysis_result = json.loads(response_json_str)

        return analysis_result

    except Exception as e:
        print(f"เรียก LLM ไม่ได้อะ dude : {e}")
        return None

def main():

    audio_file = "/content/EV Hack Video.mp4"

    if not os.path.exists(audio_file):
        print(f"!!! [Error เเล้วพรี่] !!!")
        print(f"ไม่เจอไฟล์ '{audio_file}' เช็คดีๆดิ๊")
        return

    transcript, duration = transcribe_audio(audio_file)

    if not transcript:
        print("ถอดเสียงบ่ได้")
        return

    analysis_data = analyze_presentation(transcript, duration)

    if analysis_data:
        print("\n\nผลการวิเคราะห์ (JSON Output)")

        final_json_output = {
            "file_name": audio_file,
            "analysis": analysis_data
        }

        print(json.dumps(final_json_output, indent=2, ensure_ascii=False))

        with open("analysis_result.json", "w", encoding="utf-8") as f:
            json.dump(final_json_output, f, indent=2, ensure_ascii=False)
        print("\n(บันทึกผลลัพธ์ลงใน 'analysis_result.json' เรียบร้อย)")

if __name__ == "__main__":
    main()

กำลังโหลดโมเดล ASR (large-v3)...
กำลังถอดเสียงไฟล์: /content/EV Hack Video.mp4...


/tmp/ipython-input-2308141347.py:43: FutureWarning: get_duration() keyword argument 'filename' has been renamed to 'path' in version 0.10.0.
	This alias will be removed in version 1.0.
  duration_seconds = librosa.get_duration(filename=file_path)
/tmp/ipython-input-2308141347.py:43: FutureWarning: PySoundFile failed. Trying audioread instead.
	Audioread support is deprecated in librosa 0.10.0 and will be removed in version 1.0.
  duration_seconds = librosa.get_duration(filename=file_path)


--- Transcript (ผลดิบจาก Whisper) ---
สวัสดีค่ะ พวกเรามาจากทีม by4dad ขี่ไปโซ่หลุด ทุกท่านลองในภาพตามดูเลยค่ะคุณกําลังขับรถอีบีออกไปทํางานตอนเช้า ทุกอย่างก็ดูร่ามเรื่องดีจนกระทั่งคุณสังเกตเห็นว่าแบตคุณใกล้หมดแล้ว คุณก็เลยลองเปิดแอปแผนที่เพื่อแบบหาสถานีชาร์จที่ใกล้ที่สุด แต่ข้อมูลไม่อัปเดต บางที่เต็ม บางที่ใช้ไม่ได้ทําให้คุณต้องวนหาจุดที่ชาร์จได้ ซึ่งคุณก็ต้องเสียทั้งเวลา ลองถึงแบตด้วยแบตคุณก็มีน้อยแล้ว ทําให้คุณต้องเสียเวลา ทั้งเสียแบตเพิ่มไปอีกหรือ..วันหนึ่ง.. you break in the trainแต่ว่าบ้างเอิญเกิดปัญหาการทางคุณก็เลยกำลังหาศูนย์ซ่อมแต่ nuancedแต่คุณก็ไม่แน่ใจว่าที่ไหนมันจะไว้ใจได้หรือค่าซ่อมจะสูงแค่ไหนหรือต้องรอคิวนันเท่านั้นทำให้คุณต้องเสี่ยเวลาหาศูนย์ซ่อมเองหรือการณีที่คุณกำลังจะชาร์จรถ EVแต่คุณไม่รู้ว่ามีค่าจะจ่ายประมาณเท่าไหร่เพราะแต่อาสารนี้ก็คิดราคาต่างกันบางที่แพงกว่าได้คิดหรือบางที่ไม่มีเรทราคาให้ดูเลยทำให้คุณต้องมาลุ้นตอนที่ถึงที่ชาร์จไฟจริงก่อนหรือว่าอาจจะต้องรอลุ้นอีกทีคือตอนบินออกเลยหรือคุณอยากจะช่วยลดบุญรับพิษและสับสุนพลังงานสะอาดแต่คุณไม่รู้ว่าทุกกิโลเมตรที่คุณขับ EV น

In [11]:
from groq import Groq

client =  ["GROQ_API_KEY"]

# 1. เรียก API ด้วยโมเดลใหม่
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "วิเคราะห์ Transcript นี้: [เนื้อหา Transcript ของคุณ]"
        }
    ],
    model="llama-3.3-70b-versatile", # ✅ ใช้โมเดลใหม่
)

# 2. ดึง 'content' ที่เป็น string คำตอบออกมา
analysis_result = chat_completion.choices[0].message.content

# 3. พิมพ์ผลลัพธ์เพื่อตรวจสอบ
print(analysis_result)
# ผลลัพธ์ที่ได้จะเป็น string เช่น "ผลการวิเคราะห์ Transcript คือ..."

AttributeError: 'list' object has no attribute 'chat'

In [ ]:
# (โค้ดต่อจากข้างบน)
# ... ได้ analysis_result มาแล้ว ...

# ตัวอย่างการใช้ SQL (สมมติว่าคุณเชื่อมต่อ DB ไว้แล้ว)
user_id = 123 # ID ของผู้ใช้ที่ส่งคำขอ
original_transcript = "เนื้อหา Transcript..."

# สร้างคำสั่ง SQL เพื่อ 'INSERT' ผลลัพธ์ลงไป
sql_query = """
INSERT INTO analysis_logs (user_id, transcript, llm_result, model_used)
VALUES (%s, %s, %s, %s)
"""
# ทำการ execute query
db_cursor.execute(sql_query, (user_id, original_transcript, analysis_result, "llama-3.3-70b-versatile"))
db_connection.commit() # ยืนยันการบันทึก

print("บันทึกผลการวิเคราะห์ลง Database เรียบร้อยแล้ว")

In [ ]:
from flask import Flask, request, jsonify
from groq import Groq

app = Flask(__name__)
client = Groq(api_key="YOUR_GROQ_API_KEY")

@app.route('/analyze', methods=['POST'])
def analyze_transcript():
    # 1. รับ Transcript มาจาก Frontend
    data = request.json
    transcript = data.get('transcript')

    if not transcript:
        return jsonify({"error": "No transcript provided"}), 400

    # 2. เรียก Groq LLM
    try:
        chat_completion = client.chat.completions.create(
            messages=[{"role": "user", "content": f"วิเคราะห์ Transcript นี้: {transcript}"}],
            model="llama-3.3-70b-versatile",
        )

        # 3. ดึงผลลัพธ์
        analysis_result = chat_completion.choices[0].message.content

        # 4. (ทางเลือก) บันทึกลง DB ตรงนี้
        # db.save_log(user_id=..., result=analysis_result)

        # 5. ส่งผลลัพธ์กลับไปให้ Frontend
        return jsonify({"analysis": analysis_result})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == '__main__':
    app.run(debug=True)